# Amazon Bedrock AgentCore Policy - NL2Cedar Demo

## Overview

Welcome to the Amazon Bedrock AgentCore Policy hands-on demo! This notebook will guide you through the complete workflow of generating Cedar policies from natural language. This notebook will demonstrate the various types of policies that can be generated and demonstrate how to understand various policy constuctions.

### What is Natural Language Authoring in Amazon Bedrock AgentCore Policy?

NL2Cedar - Natural Language Authoring of Cedar Policies helps you write authorization requirements in natural language which are automatically convert to Cedar syntax as well as verified that the generated policies match your requirements. 

---

## Prerequisites

Before starting, ensure you have:

- ✅ AWS CLI configured with appropriate credentials
- ✅ Python 3.10+ with boto3 installed
- ✅ `bedrock_agentcore_starter_toolkit` package installed
- ✅ Access to AWS Lambda (for target functions)

01-Getting-Started/AgentCore-Policy-Demo.ipynb sets up a Gateway for Insurance Underwriting Use case, woth 3 Lambda targets. We will use the same gateway setup. 

Let's get started! 🚀

---

# Step 0: Environment Setup

First, let's verify our environment and import necessary libraries.

In [5]:
# Import required libraries
import sys
import os
from pathlib import Path
import subprocess
import boto3
import bedrock_agentcore_starter_toolkit as ac_starter_toolkit
import json
import logging
import time
import requests

# Add the scripts directory to Python path
scripts_dir = Path.cwd() / 'scripts'
if str(scripts_dir) not in sys.path:
    sys.path.insert(0, str(scripts_dir))

# Verify region
session = boto3.Session()
region = session.region_name or 'us-east-1'

# Verify AWS credentials
try:
    sts = session.client('sts')
    identity = sts.get_caller_identity()
    print("✅ AWS Credentials Verified")
    print(f"   Account: {identity['Account']}")
    print(f"   User/Role: {identity['Arn']}")
except Exception as e:
    print(f"❌ AWS Credentials Error: {e}")
    print("   Please configure AWS CLI with: aws configure")

✅ AWS Credentials Verified
   Account: 849138760372
   User/Role: arn:aws:sts::849138760372:assumed-role/admin/bhrsrini-Isengard


### Check if Gateway Setup from 08-AgentCore-policy/01-Getting-Started/AgentCore-Policy-Demo.ipynb exists. If not, the next step will setup a gateway for insurance underwriting with 3 Lambda targets

In [6]:
# Check if Gateway for Insurance Underwriting exists

# Get the path to the Getting-Started directory
current_dir = Path.cwd()
getting_started_dir = current_dir.parent / "01-Getting-Started"
config_file = getting_started_dir / "config.json"
scripts_dir = getting_started_dir / "scripts"

print("🔍 Checking for configuration...")
print(f"Looking for: {config_file.relative_to(current_dir.parent.parent)}")

if config_file.exists():
    print("✅ Configuration file found!")
    
    # Optionally, verify it has the required fields
    import json
    try:
        with open(config_file, 'r') as f:
            config = json.load(f)
        
        required_fields = ['lambdas', 'gateway', 'region']
        missing_fields = [field for field in required_fields if field not in config]
        
        if missing_fields:
            print(f"⚠️  Warning: Config file is missing fields: {missing_fields}")
            print("   You may need to re-run the setup scripts.")
        else:
            print("✅ Configuration is complete!")
            
    except json.JSONDecodeError:
        print("⚠️  Warning: Config file exists but is not valid JSON")
        
else:
    print("❌ Configuration file not found!")
    print("\n" + "="*70)
    print("Setting up infrastructure...")
    print("="*70)
    
    # Change to the Getting-Started directory to run scripts
    os.chdir(getting_started_dir)
    
    try:
        # Step 1: Deploy Lambda functions
        print("\n📦 Step 1: Deploying Lambda functions...")
        print("-"*70)
        deploy_lambda_script = scripts_dir / "lambda-target-setup" / "deploy_lambdas.py"
        result = subprocess.run(
            [sys.executable, str(deploy_lambda_script)],
            capture_output=True,
            text=True
        )
        print(result.stdout)
        
        # Step 2: Setup Gateway
        print("\n🌐 Step 2: Setting up AgentCore Gateway...")
        print("-"*70)
        setup_gateway_script = scripts_dir / "setup_gateway.py"
        result = subprocess.run(
            [sys.executable, str(setup_gateway_script)],
            capture_output=True,
            text=True
        )
        print(result.stdout)
        
        print("\n" + "="*70)
        print("✅ Infrastructure setup complete!")
        print("="*70)
        
        # Verify config was created
        if config_file.exists():
            print(f"✅ Configuration file created: {config_file}")
        else:
            print("⚠️  Warning: Setup completed but config.json was not created")
            
    except Exception as e:
        print(f"\n❌ Setup failed: {e}")
        print("\nPlease run the setup scripts manually:")
        print(f"1. cd {getting_started_dir}")
        print(f"2. python scripts/lambda-target-setup/deploy_lambdas.py")
        print(f"3. python scripts/setup_gateway.py")
    finally:
        # Change back to original directory
        os.chdir(current_dir)

print("\n" + "="*70)

🔍 Checking for configuration...
Looking for: 08-AgentCore-policy/01-Getting-Started/config.json
❌ Configuration file not found!

Setting up infrastructure...

📦 Step 1: Deploying Lambda functions...
----------------------------------------------------------------------
🚀 Deploying Lambda Functions


🔐 No role provided, setting up IAM role...
   ✅ Using existing IAM role: AgentCoreLambdaExecutionRole

📦 Deploying ApplicationTool...
   ℹ️  Function exists, updating code...
   ✅ Code updated
   ARN: arn:aws:lambda:us-east-1:849138760372:function:ApplicationTool

📦 Deploying ApprovalTool...
   ℹ️  Function exists, updating code...
   ✅ Code updated
   ARN: arn:aws:lambda:us-east-1:849138760372:function:ApprovalTool

📦 Deploying RiskModelTool...
   ℹ️  Function exists, updating code...
   ✅ Code updated
   ARN: arn:aws:lambda:us-east-1:849138760372:function:RiskModelTool


💾 Configuration saved to: /Users/bhrsrini/projects/Agentcore/Policy/amazon-bedrock-agentcore-samples/01-tutorials/08-Ag

---

# Step 1: Create a Policy Engine

Now we'll create a Policy Engine to hold our Cedar policies

A Policy Engine is a collection of policies. It can be associated with a gateway for real-time enforcement of policies on the inbound traffic.
We will be creating policies in this policy engine in the upcoming steps

### Create Policy Engine

First, we'll create a Policy Engine to hold our Cedar policies:

In [ ]:
from bedrock_agentcore_starter_toolkit.operations.policy.client import PolicyClient

policy_client = PolicyClient(region_name=region)
policy_client.logger.setLevel(logging.INFO)

# Create a Policy Engine
print("🔧 Creating Policy Engine...")
engine = policy_client.create_or_get_policy_engine(
    name="InsurancePolicyEngine_NL2Cedar",
    description="Policy engine for insurance underwriting governance"
)
print(f"✓ Policy Engine: {engine['policyEngineId']}\n")

# Save Policy Engine in the configuration file
with open(config_file, "r") as f:
    config = json.load(f)

# Add policy engine information (without removing existing data)
config["policy_engine_id"] = engine["policyEngineId"]
config["policy_engine_arn"] = engine["policyEngineArn"]

# Write back the updated config
with open(config_file, "w") as f:
    json.dump(config, f, indent=2)

2025-12-06 21:06:46,014 - bedrock_agentcore.policy - INFO - Creating or getting Policy Engine: InsurancePolicyEngine_NL2Cedar


🔧 Creating Policy Engine...


2025-12-06 21:06:46,312 - bedrock_agentcore.policy - INFO - Creating Policy Engine: InsurancePolicyEngine_NL2Cedar
2025-12-06 21:06:46,460 - bedrock_agentcore.policy - INFO - ✓ Policy Engine creation initiated: arn:aws:bedrock-agentcore:us-east-1:849138760372:policy-engine/InsurancePolicyEngine_NL2Cedar-mwqd6clzpg
2025-12-06 21:06:46,461 - bedrock_agentcore.policy - INFO - Waiting for Policy Engine to be active...
2025-12-06 21:06:48,666 - bedrock_agentcore.policy - INFO - ✓ Policy Engine is active


✓ Policy Engine: InsurancePolicyEngine_NL2Cedar-mwqd6clzpg



FileNotFoundError: [Errno 2] No such file or directory: 'config.json'

## Step 2: Generate policy from natural language

Now we will generate a Cedar policy from natural language using NL2Cedar capability. The policy creation using NL2Cedar involves two steps: first, the generation of a Cedar policy from natural language, and then creation of the policy in the Policy engine. 

> **💡 Tip**: The schema of the targets on the Gateway are provided to the NL2Cedar capability to help the foundation model understand the names of the targets and the parameters.

The Gateway has 3 Lambda targets:
1. Application Tool: Simplified Application Creation (Mocked Up for Demo purpose)
 Creates insurance applications with applicant region and coverage amount
 Parameters:
 - applicant_region: Customer's geographic region
 - coverage_amount: Requested insurance coverage amount

2. Risk Model Tool: Simplified Risk Model Access (Mocked Up for Demo purpose)
 Invokes risk scoring model and returns assessment
 Parameters:
 - API_classification: API classification (public, internal, restricted)
 - data_governance_approval: Whether data governance has approved model usage

3. Approval Tool: - Insurance Approval Process (Mocked Up for Demo purpose)
 Approves underwriting decisions and claim amounts
 Parameters:
 - claim_amount: Insurance claim/coverage amount
 - risk_level: Risk level assessment (low, medium, high, critical)


 Our natural language statements can refer to any of these targets and introduce constraints on agent-tool access based on these parameters. 

In [12]:
# Create Cedar policy
print("\n📝 Generating Cedar Policy from Natural language...")
print("\n📝 Simple natural language statement")

nl_input = "Allow all users to invoke the application tool when the coverage amount is under 1 million and the application region is US or CAN"

result = policy_client.generate_policy(
    policy_engine_id=config["policy_engine_id"],
    name=f"nl_policy_{int(time.time())}",
    resource={"arn": config['gateway']['gateway_arn']},
    content={"rawText": nl_input},
    fetch_assets=True
    )

2025-12-06 21:23:55,921 - bedrock_agentcore.policy - INFO - Generating policies from natural language: nl_policy_1765085035
2025-12-06 21:23:55,921 - bedrock_agentcore.policy - INFO - Starting Policy Generation: nl_policy_1765085035



📝 Generating Cedar Policy from Natural language...

📝 Simple natural language statement


2025-12-06 21:23:56,554 - bedrock_agentcore.policy - INFO - ✓ Policy Generation initiated: arn:aws:bedrock-agentcore:us-east-1:849138760372:policy-engine/InsurancePolicyEngine_NL2Cedar-mwqd6clzpg/policy-generation/nl_policy_1765085035-bw3qy6xota
2025-12-06 21:23:56,555 - bedrock_agentcore.policy - INFO - Started generation nl_policy_1765085035-bw3qy6xota, waiting for completion...
2025-12-06 21:23:56,694 - bedrock_agentcore.policy - INFO - Generation in progress (attempt 1/30)...
2025-12-06 21:23:58,831 - bedrock_agentcore.policy - INFO - Generation in progress (attempt 2/30)...
2025-12-06 21:24:00,939 - bedrock_agentcore.policy - INFO - Generation in progress (attempt 3/30)...
2025-12-06 21:24:03,085 - bedrock_agentcore.policy - INFO - Generation in progress (attempt 4/30)...
2025-12-06 21:24:05,194 - bedrock_agentcore.policy - INFO - Generation in progress (attempt 5/30)...
2025-12-06 21:24:07,306 - bedrock_agentcore.policy - INFO - ✓ Policy generation complete
2025-12-06 21:24:09,31

In [14]:
if result.get('status') == 'GENERATED' and result.get('generatedPolicies'):
    generated_policy = result['generatedPolicies'][0]
    cedar_statement = generated_policy.get('definition', {}).get('cedar', {}).get('statement', 'N/A')
    
    print("Generated Cedar Policy:")
    print("=" * 60)
    print(cedar_statement)
    print("=" * 60)

Generated Cedar Policy:
permit(
  principal,
  action == AgentCore::Action::"ApplicationToolTarget___create_application",
  resource == AgentCore::Gateway::"arn:aws:bedrock-agentcore:us-east-1:849138760372:gateway/gw-insurance-underwriting-l8uywbounk"
) when {
  (((context.input).coverage_amount) < 1000000) && ((((context.input).applicant_region) == "US") || (((context.input).applicant_region) == "CAN"))
};


## Step 2: Create a policy from the generated Cedar policy

In [15]:
if result.get('status') == 'GENERATED' and result.get('generatedPolicies'):
    generated_policy = result['generatedPolicies'][0]
    application_creation_policy = policy_client.create_policy(
        policy_engine_id=config["policy_engine_id"],
        name="application_creation_policy",
        description="Allow application creation when coverage is under $1M and region is US or CA",
        definition=generated_policy.get('definition', {})
    )

2025-12-06 21:28:54,354 - bedrock_agentcore.policy - INFO - Creating Policy: application_creation_policy
2025-12-06 21:28:54,685 - bedrock_agentcore.policy - INFO - ✓ Policy creation initiated: arn:aws:bedrock-agentcore:us-east-1:849138760372:policy-engine/InsurancePolicyEngine_NL2Cedar-mwqd6clzpg/policy/application_creation_policy-hsprx_gvle


---

# Other types of policy generations from natural language

### 1. Multi-line statements
When multi-line statements are provided, multiple policies will be generated and present in the result['generatedPolicies']. The consistently appearing delimited (be it comma, full stop, semi-colon, etc) will be picked up to distinguish between individual policy statements


In [16]:
print("\n📝 Multi-line statement")

nl_input = """Allow all users to invoke the risk model tool when data governance approval is true. 
Block users from calling the application tool unless coverage amount is present"""

result = policy_client.generate_policy(
    policy_engine_id=config["policy_engine_id"],
    name=f"nl_policy_{int(time.time())}",
    resource={"arn": config['gateway']['gateway_arn']},
    content={"rawText": nl_input},
    fetch_assets=True
    )

if result.get('status') == 'GENERATED' and result.get('generatedPolicies'):
    for generated_policy in result['generatedPolicies']:

        cedar_statement = generated_policy.get('definition', {}).get('cedar', {}).get('statement', 'N/A')
        
        print("Generated Cedar Policy:")
        print("=" * 60)
        print(cedar_statement)
        print("=" * 60)

2025-12-06 21:35:15,797 - bedrock_agentcore.policy - INFO - Generating policies from natural language: nl_policy_1765085715
2025-12-06 21:35:15,798 - bedrock_agentcore.policy - INFO - Starting Policy Generation: nl_policy_1765085715



📝 Multi-line statement


2025-12-06 21:35:16,506 - bedrock_agentcore.policy - INFO - ✓ Policy Generation initiated: arn:aws:bedrock-agentcore:us-east-1:849138760372:policy-engine/InsurancePolicyEngine_NL2Cedar-mwqd6clzpg/policy-generation/nl_policy_1765085715-j9k01u8qxy
2025-12-06 21:35:16,508 - bedrock_agentcore.policy - INFO - Started generation nl_policy_1765085715-j9k01u8qxy, waiting for completion...
2025-12-06 21:35:16,666 - bedrock_agentcore.policy - INFO - Generation in progress (attempt 1/30)...
2025-12-06 21:35:18,822 - bedrock_agentcore.policy - INFO - Generation in progress (attempt 2/30)...
2025-12-06 21:35:20,938 - bedrock_agentcore.policy - INFO - Generation in progress (attempt 3/30)...
2025-12-06 21:35:23,061 - bedrock_agentcore.policy - INFO - Generation in progress (attempt 4/30)...
2025-12-06 21:35:25,188 - bedrock_agentcore.policy - INFO - Generation in progress (attempt 5/30)...
2025-12-06 21:35:27,318 - bedrock_agentcore.policy - INFO - Generation in progress (attempt 6/30)...
2025-12-06

Generated Cedar Policy:
permit(
  principal,
  action == AgentCore::Action::"RiskModelToolTarget___invoke_risk_model",
  resource == AgentCore::Gateway::"arn:aws:bedrock-agentcore:us-east-1:849138760372:gateway/gw-insurance-underwriting-l8uywbounk"
) when {
  ((context.input).data_governance_approval) == true
};
Generated Cedar Policy:
forbid(
  principal,
  action == AgentCore::Action::"ApplicationToolTarget___create_application",
  resource == AgentCore::Gateway::"arn:aws:bedrock-agentcore:us-east-1:849138760372:gateway/gw-insurance-underwriting-l8uywbounk"
) when {
  !(!(((context.input).coverage_amount) <= 0))
};


### 2. Principal Related Policy Statements
It is possible to create policies that assert conditions based on the principal scope, which is relayed through the idP representation in the form of the OAuth access token. As you can configure what attributes are stored in the OAuth token which can be custom, for NL2Cedar generation, providing the exact tag would help NL2Cedar create the correct Cedar policy

In [18]:
print("\n📝 Principal Scope statements")

nl_inputs = [
    "Allow principals with username \"test-user\" to invoke the risk model tool",
    str("Forbid principals to access the approval tool unless they have the scope group:Controller <idp_claims>[\"scope\"]</idp_claims>"),
    str("Block principals from using risk model tool and approval tool unless the principal has role \"senior-adjuster\"")
]

for nl_input in nl_inputs:
    result = policy_client.generate_policy(
        policy_engine_id=config["policy_engine_id"],
        name=f"nl_policy_{int(time.time())}",
        resource={"arn": config['gateway']['gateway_arn']},
        content={"rawText": nl_input},
        fetch_assets=True
        )

    if result.get('status') == 'GENERATED' and result.get('generatedPolicies'):
        for generated_policy in result['generatedPolicies']:

            cedar_statement = generated_policy.get('definition', {}).get('cedar', {}).get('statement', 'N/A')
            print("=" * 60)
            print(f"Natural Language: {nl_input}")
            print("Generated Cedar Policy:")
            print("=" * 60)
            print(cedar_statement)
            print("=" * 60)

2025-12-06 21:56:20,765 - bedrock_agentcore.policy - INFO - Generating policies from natural language: nl_policy_1765086980
2025-12-06 21:56:20,767 - bedrock_agentcore.policy - INFO - Starting Policy Generation: nl_policy_1765086980



📝 Principal Scope statements


2025-12-06 21:56:21,419 - bedrock_agentcore.policy - INFO - ✓ Policy Generation initiated: arn:aws:bedrock-agentcore:us-east-1:849138760372:policy-engine/InsurancePolicyEngine_NL2Cedar-mwqd6clzpg/policy-generation/nl_policy_1765086980-tplf7n30sk
2025-12-06 21:56:21,420 - bedrock_agentcore.policy - INFO - Started generation nl_policy_1765086980-tplf7n30sk, waiting for completion...
2025-12-06 21:56:21,523 - bedrock_agentcore.policy - INFO - Generation in progress (attempt 1/30)...
2025-12-06 21:56:23,623 - bedrock_agentcore.policy - INFO - Generation in progress (attempt 2/30)...
2025-12-06 21:56:25,722 - bedrock_agentcore.policy - INFO - Generation in progress (attempt 3/30)...
2025-12-06 21:56:27,826 - bedrock_agentcore.policy - INFO - Generation in progress (attempt 4/30)...
2025-12-06 21:56:29,961 - bedrock_agentcore.policy - INFO - Generation in progress (attempt 5/30)...
2025-12-06 21:56:32,072 - bedrock_agentcore.policy - INFO - ✓ Policy generation complete
2025-12-06 21:56:34,07

Natural Language: Allow principals with username "test-user" to invoke the risk model tool
Generated Cedar Policy:
permit(
  principal,
  action == AgentCore::Action::"RiskModelToolTarget___invoke_risk_model",
  resource == AgentCore::Gateway::"arn:aws:bedrock-agentcore:us-east-1:849138760372:gateway/gw-insurance-underwriting-l8uywbounk"
) when {
  (principal.hasTag("username")) && ((principal.getTag("username")) == "test-user")
};


2025-12-06 21:56:34,487 - bedrock_agentcore.policy - INFO - ✓ Policy Generation initiated: arn:aws:bedrock-agentcore:us-east-1:849138760372:policy-engine/InsurancePolicyEngine_NL2Cedar-mwqd6clzpg/policy-generation/nl_policy_1765086994-50hvk6x2ps
2025-12-06 21:56:34,488 - bedrock_agentcore.policy - INFO - Started generation nl_policy_1765086994-50hvk6x2ps, waiting for completion...
2025-12-06 21:56:34,581 - bedrock_agentcore.policy - INFO - Generation in progress (attempt 1/30)...
2025-12-06 21:56:36,686 - bedrock_agentcore.policy - INFO - Generation in progress (attempt 2/30)...
2025-12-06 21:56:38,814 - bedrock_agentcore.policy - INFO - Generation in progress (attempt 3/30)...
2025-12-06 21:56:40,915 - bedrock_agentcore.policy - INFO - Generation in progress (attempt 4/30)...
2025-12-06 21:56:43,050 - bedrock_agentcore.policy - INFO - Generation in progress (attempt 5/30)...
2025-12-06 21:56:45,157 - bedrock_agentcore.policy - INFO - Generation in progress (attempt 6/30)...
2025-12-06

Natural Language: Forbid principals to access the approval tool unless they have the scope group:Controller <idp_claims>["scope"]</idp_claims>
Generated Cedar Policy:
forbid(
  principal,
  action == AgentCore::Action::"ApprovalToolTarget",
  resource == AgentCore::Gateway::"arn:aws:bedrock-agentcore:us-east-1:849138760372:gateway/gw-insurance-underwriting-l8uywbounk"
) when {
  !((principal.hasTag("scope")) && ((principal.getTag("scope")) like "*group:Controller*"))
};


2025-12-06 21:56:53,924 - bedrock_agentcore.policy - INFO - ✓ Policy Generation initiated: arn:aws:bedrock-agentcore:us-east-1:849138760372:policy-engine/InsurancePolicyEngine_NL2Cedar-mwqd6clzpg/policy-generation/nl_policy_1765087013-vfwk0l3hxy
2025-12-06 21:56:53,924 - bedrock_agentcore.policy - INFO - Started generation nl_policy_1765087013-vfwk0l3hxy, waiting for completion...
2025-12-06 21:56:54,021 - bedrock_agentcore.policy - INFO - Generation in progress (attempt 1/30)...
2025-12-06 21:56:56,145 - bedrock_agentcore.policy - INFO - Generation in progress (attempt 2/30)...
2025-12-06 21:56:58,247 - bedrock_agentcore.policy - INFO - Generation in progress (attempt 3/30)...
2025-12-06 21:57:00,348 - bedrock_agentcore.policy - INFO - Generation in progress (attempt 4/30)...
2025-12-06 21:57:02,454 - bedrock_agentcore.policy - INFO - Generation in progress (attempt 5/30)...
2025-12-06 21:57:04,555 - bedrock_agentcore.policy - INFO - ✓ Policy generation complete
2025-12-06 21:57:06,56

Natural Language: Block principals from using risk model tool and approval tool unless the principal has role "senior-adjuster"
Generated Cedar Policy:
forbid(
  principal,
  action in [AgentCore::Action::"RiskModelToolTarget",AgentCore::Action::"ApprovalToolTarget"],
  resource == AgentCore::Gateway::"arn:aws:bedrock-agentcore:us-east-1:849138760372:gateway/gw-insurance-underwriting-l8uywbounk"
) when {
  !((principal.hasTag("role")) && ((principal.getTag("role")) == "senior-adjuster"))
};


# CleanUp

In [20]:
from bedrock_agentcore_starter_toolkit.operations.gateway.client import GatewayClient
from bedrock_agentcore_starter_toolkit.operations.policy.client import PolicyClient

with open(config_file, "r") as f:
    config = json.load(f)

# Clean up Policy Engine first
print("🧹 Cleaning up Policy Engine...")
policy_client = PolicyClient(region_name=config["region"])
policy_client.cleanup_policy_engine(config["policy_engine_id"])
print("✓ Policy Engine cleaned up\n")

# Then clean up Gateway
print("🧹 Cleaning up Gateway...")
gateway_client = GatewayClient(region_name=config["region"])
gateway_client.cleanup_gateway(config['gateway']["gateway_id"], config['gateway']["client_info"])
print("✅ Cleanup complete!")

2025-12-06 21:58:11,942 - bedrock_agentcore.policy - INFO - 🧹 Cleaning up Policy Engine: InsurancePolicyEngine_NL2Cedar-mwqd6clzpg


🧹 Cleaning up Policy Engine...


2025-12-06 21:58:12,231 - bedrock_agentcore.policy - INFO - Found 1 policies to delete
2025-12-06 21:58:12,232 - bedrock_agentcore.policy - INFO -   • Deleting policy: application_creation_policy
2025-12-06 21:58:12,232 - bedrock_agentcore.policy - INFO - Deleting Policy: application_creation_policy-hsprx_gvle
2025-12-06 21:58:12,382 - bedrock_agentcore.policy - INFO - ✓ Policy deletion initiated: application_creation_policy-hsprx_gvle
2025-12-06 21:58:12,383 - bedrock_agentcore.policy - INFO -     ✓ Policy deletion initiated: application_creation_policy
2025-12-06 21:58:16,690 - bedrock_agentcore.policy - INFO -     ✓ Policy deleted
2025-12-06 21:58:16,691 - bedrock_agentcore.policy - INFO -   • Deleting policy engine: InsurancePolicyEngine_NL2Cedar-mwqd6clzpg
2025-12-06 21:58:16,692 - bedrock_agentcore.policy - INFO - Deleting Policy Engine: InsurancePolicyEngine_NL2Cedar-mwqd6clzpg
2025-12-06 21:58:16,819 - bedrock_agentcore.policy - INFO - ✓ Policy Engine deletion initiated: Insura

✓ Policy Engine cleaned up

🧹 Cleaning up Gateway...


2025-12-06 21:58:17,134 - bedrock_agentcore.gateway - INFO -     Found 3 targets to delete
2025-12-06 21:58:17,134 - bedrock_agentcore.gateway - INFO -   • Deleting target: 03KDIF4J0U
2025-12-06 21:58:17,304 - bedrock_agentcore.gateway - INFO -     ✓ Target deletion initiated: 03KDIF4J0U
2025-12-06 21:58:22,310 - bedrock_agentcore.gateway - INFO -   • Deleting target: M5LMBYFSLS
2025-12-06 21:58:22,522 - bedrock_agentcore.gateway - INFO -     ✓ Target deletion initiated: M5LMBYFSLS
2025-12-06 21:58:27,528 - bedrock_agentcore.gateway - INFO -   • Deleting target: RTUSAOCBW1
2025-12-06 21:58:27,710 - bedrock_agentcore.gateway - INFO -     ✓ Target deletion initiated: RTUSAOCBW1
2025-12-06 21:58:32,712 - bedrock_agentcore.gateway - INFO -   • Verifying targets deletion...
2025-12-06 21:58:37,870 - bedrock_agentcore.gateway - INFO -     ✓ All targets deleted
2025-12-06 21:58:37,873 - bedrock_agentcore.gateway - INFO -   • Deleting gateway: gw-insurance-underwriting-l8uywbounk
2025-12-06 21

✅ Cleanup complete!
